In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math

from oracles import LogReg
from methods import Standard_Newton, Newton_Star
from methods import NL1
from methods import DINGO, Gradient_Descent
from methods import diana, adiana
from methods import FedNL, FedNL_CR, FedNL_LS
from methods import Newton_Zero
from methods import FedNL_BC, FedNL_PP
from methods import Artemis, DORE

from utils import read_data, generate_synthetic
from utils import loss_logistic, grad, random_k, positive_part
from utils import default_dataset_parameters
from utils import topK_vectors, biased_rounding, randomK_vectors
from utils import Low_Rank, PowerSgdCompression, TopK
from utils import pos_projection, semidef_projection
from utils import random_spars_matrix, rand_dith

from easydict import EasyDict

In [ ]:
data_name = 'a1a'
dataset_path = './Datasets/{}.txt'.format(data_name)


# regularization parameter
lmb = 1e-3

# number of nodes, size of local data, and dimension of the problem
# according to the paper
N = default_dataset_parameters[data_name]['N']# size of the whole data set
n = default_dataset_parameters[data_name]['n']# number of nodes
m = default_dataset_parameters[data_name]['m']# size of local data set
d = default_dataset_parameters[data_name]['d']# dimension of the problem

In [ ]:
A, b = read_data(dataset_path=dataset_path, 
                 N=N, n=n, m=m, d=d, lmb=lmb,
                labels=['+1', '-1'])

In [ ]:
# set the problem 
logreg = LogReg(A=A, b=b, reg_coef=lmb, n=n, m=m)

# find the solution using Newton's method starting from zeros for 20 iterations
Newton = Standard_Newton(logreg)
Newton.find_optimum(x0=np.zeros(d), n_steps=20,verbose=True)

In [ ]:
# set optimum and optimal function value
x_opt = logreg.get_optimum()
f_opt = logreg.function_value(x_opt)

# Newton-Zero

***input:***
- `x` - initial model weightsn
- `init_cost` - if True, then the communication cost of initalization is inclued
- `line_search` - if True, then the method uses backtracking line search procedure
- `gamma` - parameter of line search procedure
- `c` - parameter of line search procedure
- `tol` - desired tolerance of the solution
- `max_iter` - maximum number of iterations
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.1

# define the method
n0 = Newton_Zero(logreg)

# run the method
fv, bi, it = n0.method(x=x, init_cost=False,
                      line_search=False, gamma=0.5, c=0.5,
                      tol=1e-15, max_iter=100, verbose=True)

# FedNL

the example how to run FedNL method. It has the following parameters:

***input:***
- `x` - initial point
- `hes_comp_param` - parameter of local Hessian compression operator
- `hes_comp_name` - name of compression operator for Hessians
  - `LowRank` (Rank-R) compression operator (`upd_rule` 1 or 2)
  - `TopK` (Top-K) compression operator (`upd_rule` 1 or 2)
  - `PowerSGD` compression operator (`upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compression operator (`upd_rule` 3)
- `init` - if zero, then $\mathbf{H}^0_i = \mathbf{0}$, otherwise $\mathbf{H}^0_i = \nabla^2 f_i(x^0)$
- `init_cost` - if True, then the communication cost on initialization is included
- `option` 
  - if 1, then the method uses Option 1 of FedNL 
  - if 2, then the method uses Option 2 of FedNL
- `upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize $\alpha=1-\sqrt{1-\delta}$
  - if 2, then method requires Biased compressor and uses stepsize $\alpha=1$
  - if 3, then methods requires Unbiased compressor and uses stepsize $\alpha=\frac{1}{\omega+1}$
- `lr` - learning rate if the user wants to use PowerSGD where stepsize can be chosen
- `max_iter` - maximum number of iterations
- `tol` - desired tolerance of the solution
- `bits_compute` 
  - if OneSided, then bits are computed in upside direction only
  - if TwoSided, then bits are computed in both directions: upside and backside
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
np.random.seed(1)
x = x_opt + np.ones(d)*0.15

# define the method
fednl = FedNL(logreg)

# run the method
fv, bi, it = fednl.method(x=x,hes_comp_name='LowRank', hes_comp_param=1, 
                          init='nonzero', init_cost=True,
                          option=1, upd_rule=2,
                          lr=None, max_iter=1000, tol=1e-15,
                          bits_compute='OneSided', verbose=True)

# FedNL-LS

the example how to run FedNL method. It has the following parameters:

***input:***
- `x` - initial point
- `hes_comp_param` - parameter of local Hessian compression operator. It can be
- `hes_comp_name` - name of compression operator for Hessians
  - `LowRank` (Rank-R) compression operator (`upd_rule` 1 or 2)
  - `TopK` (Top-K) compression operator (`upd_rule` 1 or 2)
  - `PowerSGD` compression operator (`upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compression operator (`upd_rule` 3)
- `init` - if zero, then $\mathbf{H}^0_i = \mathbf{0}$, otherwise $\mathbf{H}^0_i = \nabla^2 f_i(x^0)$
- `init_cost` - if True, then the communication cost on initialization is included
- `max_iter` - maximum number of iterations
- `tol` - desired tolerance of the solution
- `bits_compute` 
  - if OneSided, then bits are computed in upside direction only
  - if TwoSided, then bits are computed in both directions: upside and backside
- `upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize $\alpha=1-\sqrt{1-\delta}$
  - if 2, then method requires Biased compressor and uses stepsize $\alpha=1$
  - if 3, then methods requires Unbiased compressor and uses stepsize $\alpha=\frac{1}{\omega+1}$
- `gamma` - parameter of line search procedure
- `c` - parameter of line search procedure
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
x  = x_opt + np.ones(d)*2

# define method
fednl_ls = FedNL_LS(logreg)

# run the method

fv, bi, it = fednl_ls.method(x=x, hes_comp_param=d, hes_comp_name='TopK', 
                             init='nonzero', init_cost=False, 
                             max_iter=20, tol=1e-15, 
                             bits_compute='OneSided', upd_rule=1,
                             gamma=0.5, c=0.5, verbose=True)

# FedNL-CR

***input:***
- `x` - initial point
- `hes_comp_param` - parameter of local Hessian compression operator
- `hes_comp_name` - name of compression operator for Hessians
  - `LowRank` (Rank-R) compression operator (`upd_rule` 1 or 2)
  - `TopK` (Top-K) compression operator (`upd_rule` 1 or 2)
  - `PowerSGD` compression operator (`upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compression operator (`upd_rule` 3)
- `init` - if zero, then $\mathbf{H}^0_i = \mathbf{0}$, otherwise $\mathbf{H}^0_i = \nabla^2 f_i(x^0)$
- `init_cost` - if True, then the communication cost on initialization is included
- `upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize $\alpha=1-\sqrt{1-\delta}$
  - if 2, then method requires Biased compressor and uses stepsize $\alpha=1$
  - if 3, then methods requires Unbiased compressor and uses stepsize $\alpha=\frac{1}{\omega+1}$
- `max_iter` - maximum number of iterations
- `tol` - desired tolerance of the solution
- `bits_compute` 
  - if OneSided, then bits are computed in upside direction only
  - if TwoSided, then bits are computed in both directions: upside and backside
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
x  = x_opt + np.ones(d)*2

# define method
fednl_cr = FedNL_CR(logreg)

# run the method

fv, bi, it = fednl_cr.method(x=x, hes_comp_param=1, hes_comp_name='LowRank', 
                             init='zero', init_cost=False, 
                             upd_rule=3, max_iter=20, tol=1e-15, 
                             bits_compute='OneSided', verbose=True)

# FedNL-BC

***input:***
- `x` - initial point
- `hes_comp_param` - parameter of local Hessian compression operator
- `hes_comp_name` - name of compression operator for Hessians
  - `LowRank` (Rank-R) compression operator (`upd_rule` 1 or 2)
  - `TopK` (Top-K) compression operator (`upd_rule` 1 or 2)
  - `PowerSGD` compression operator (`upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compression operator (`upd_rule` 3)
- `iter_comp_param` - parameter of compression operator applied to models
- `iter_comp_name` - name of compression operator for models
  - `TopK` (Top-K) compression operator (`iter_upd_rule` 1 or 2)
  - `Rounding` compression operator (`iter_upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compressio operator (`iter_upd_rule` 3)
- `init` - if zero, then $\mathbf{H}^0_i = \mathbf{0}$, otherwise $\mathbf{H}^0_i = \nabla^2 f_i(x^0)$
- `init_cost` - if True, then the communication cost on initialization is included
- `upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize $\alpha=1-\sqrt{1-\delta}$
  - if 2, then method requires Biased compressor and uses stepsize $\alpha=1$
  - if 3, then methods requires Unbiased compressor and uses stepsize $\alpha=\frac{1}{\omega+1}$
- `iter_upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize $\eta=1-\sqrt{1-\delta_M}$
  - if 2, then method requires Biased compressor and uses stepsize $\eta=1$
  - if 3, then methods requires Unbiased compressor and uses stepsize $\eta=\frac{1}{\omega_M+1}$
- `option` 
  - if 1, then the method uses Option 1 of FedNL-BC 
  - if 2, then the method uses Option 2 of FedNL-BC
- `max_iter` - maximum number of iterations
- `tol` - desired tolerance of the solution
- `bits_compute` 
  - if OneSided, then bits are computed in upside direction only
  - if TwoSided, then bits are computed in both directions: upside and backside
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
np.random.seed(1)
x = x_opt + np.ones(d)*0.1

# define method
fednl_bc = FedNL_BC(logreg)

# run the method
p = 0.5
fv, bi, it = fednl_bc.method(x=x, hes_comp_param=int(p*d), hes_comp_name='TopK', 
                             p=0.5, iter_comp_param=int(p*d), iter_comp_name='TopK',
                             init='nonzero', init_cost=False, 
                             upd_rule=2, iter_upd_rule=1,
                             max_iter=20, tol=1e-15, 
                             verbose=True, option=2)

# FedNL-PP

***input:*** 
- `x` - initial point
- `tau` - number of active nodes
- `hes_comp_param` - parameter of local Hessian compression operator
- `hes_comp_name` - name of compression operator for Hessians
  - `LowRank` (Rank-R) compression operator (`upd_rule` 1 or 2)
  - `TopK` (Top-K) compression operator (`upd_rule` 1 or 2)
  - `PowerSGD` compression operator (`upd_rule` 1 or 2)
  - `RandomK` (Rand-K) compression operator (`upd_rule` 3)
- `init` - if zero, then $\mathbf{H}^0_i = \mathbf{0}$, otherwise $\mathbf{H}^0_i = \nabla^2 f_i(x^0)$
- `init_cost` - if True, then the communication cost on initialization is included  
- `upd_rule` 
  - if 1, then method requires Biased compressor and uses stepsize alpha=1-\sqrt{1-delta}
  - if 2, then method requires Biased compressor and uses stepsize alpha=1
  - if 3, then methods requires Unbiased compressor and uses stepsize alpha=1/(omega+1)
- `max_iter` - maximum number of iterations
- `tol` - desired tolerance of the solution
- `bits_compute`
  - if OneSided, then bits are computed in upside direction only
  - if TwoSided, then bits are computed in both directions: upside and backside

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
np.random.seed(1)
x = x_opt + np.ones(d)*0.1

# define method
fednl_pp = FedNL_PP(logreg)

# run the method
p = 0.5
fv, bi, it = fednl_pp.method(x=x, tau=int(0.5*n),
                             hes_comp_param=1, hes_comp_name='LowRank',
                             init='zero', init_cost=False, 
                             upd_rule=2, max_iter=20, tol=1e-15, 
                             verbose=True)

# Gradient descent

***input:***
- `x` - initial model weightsn
- `line_search` - if True, then the method uses backtracking line search procedure
- `gamma` - parameter of line search procedure
- `c` - parameter of line search procedure
- `tol` - desired tolerance of the solution
- `max_iter` - maximum number of iterations
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point
x = x_opt + np.ones(d)*2

# define method
gd = Gradient_Descent(logreg)

# run the method
fv, bi, it = gd.method(x=x, line_search=True,
                       gamma=0.5, c=0.5,
                       tol=1e-15, max_iter=20,
                       verbose=True)

# DINGO

***input:***
- `x` - initial point
- `max_iter` - maximum iterations of the method
- `tol` - desired tolerance of the solution
- `phi`, `theta`, `rho` - parameters of DINGO
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
x = x_opt + np.ones(d)*0.1

# define method
dingo = DINGO(logreg)

# run the method
fv, bi = dingo.method(x=x, phi=1e-6, theta=1e-4, 
                      rho=1e-4, tol=1e-15, 
                      max_iter=20, verbose=True)

# ADIANA and DIANA

In [ ]:
# eta, alpha, theta_1, theta_2, gamma are parameters of ADIANA 
class args(EasyDict):
    def __init__(self, data_name, T, node, L, lamda, eta=0.05,
                 alpha=0.5, theta_1=0.25, theta_2=0.5, gamma=0.5,
                 beta=0.95,
                 prob=1,
                 comp_method='no_comp',
                 r=None, s=None, s_level=10):
        super().__init__()
        self.data_name = data_name
        # T - maximum number of iterations
        self.T = T
        # number of nodes
        self.node = node
        # parameters of ADIANA
        self.eta = eta
        self.alpha = alpha
        self.theta_1 = theta_1
        self.theta_2 = theta_2
        self.gamma = gamma
        self.beta = beta
        self.prob = prob
        # Lipshitz constant of gradients
        self.L = L
        # regularization parameter
        self.lamda = lamda
        # name of compression operator
        self.comp_method = comp_method
        # parameter of random sparsification
        # compression operator
        self.r = r
        # parameter of randim dithering
        # compression operator
        self.s = s
    
# find Lipshitz constant of gradients
H = np.dot(A.T,A)/N
temp = np.linalg.eigvalsh(H)
L = np.abs(temp[-1])/4

# define maximum number of iterations
max_iter = 200

# set class containing all parameters
arg = args(data_name, max_iter, n, L, lmb)

# set parameters of compression operators
arg.r = d/4
arg.s = np.sqrt(d)

# set compression operator
comp_methods = [
     'rand_dithering'
]

# put compression operator to arg's
arg.comp_method = comp_methods[0]

In [ ]:
# run ADIANA
loss_adiana, com_bits_adiana = adiana(A, b, x, arg, f_opt=f_opt, tol=1e-15)

# method prints function values each 1000 iterations

In [ ]:
# run ADIANA

loss_diana, com_bits_diana = diana(A, b, x, arg, f_opt=f_opt, tol=1e-15)
# method prints function values each 1000 iterations

# Artemis 

***input:***
- `x` - initial model weights
- `tau` - number of active nodes
- `compression` 
  - if OneSided, then random dithering is applied only on uplink direction
  - if TwoSided, then random dithering is applied both on uplink and downlink directions
- `tol` - desired tolerance of the solution
- `max_iter` - maximum number of steps of the method 
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates- - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
artemis = Artemis(logreg)

In [ ]:
fv, bi, it = artemis.method(x=x_opt+np.ones(d)*0.1, tau=n, max_iter=20)

# DORE

***input:***
- `x` - initial model weights
- `tol` - desired tolerance of the solution
- `max_iter` - maximum number of steps of the method 
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `iterates` - numpy array containing distances from current point to the solution
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
dore = DORE(logreg)

fv, bi, it = dore.method(x=x_opt+np.ones(d)*0.1, max_iter=20)

# Synthetic data

In our experiments we also generate synthetic data. This can be done using the function `generate_synthetic`. It has the following parameters:

***input:***
- `alpha`, `beta` - parameters of data
- `iid`
  - if 1, then the data distribution over nodes is iid
  - if 0, then the data distribution over nodes is non-iid
- `d` - dimension of the problem
- `n` - number of nodes
- `m` - size of local data
    
***output:***
- numpy arrays A (features) and b (labels)

Example of IID data generation:

In [ ]:
A, b = generate_synthetic(alpha=0.5, beta=0.5, iid=1, d=d, n=n, m=m)

Example of non-IID data generation:

In [ ]:
A, b = generate_synthetic(alpha=0.5, beta=0.5, iid=0, d=d, n=n, m=m)